In [6]:
import qiskit.qasm2
from qiskit.circuit import QuantumCircuit
 
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])
# dumps generates the QASM code for the circuit
print(qiskit.qasm2.dumps(qc))

# remember to eliminate imports before running on Quokka 
# send as a json file to the Quokka API

OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0],q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];


In [7]:
qiskit.qasm2.dump(qc, "myfile.qasm")

In [8]:
import qiskit.qasm2
circuit = qiskit.qasm2.load("myfile.qasm")
print(circuit)

     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 


In [9]:
# the usual imports
import qiskit
import qiskit.qasm2
import numpy as np
import requests
import os
import json
#from google.colab import files

# current Quokka address:
# req_str_qasm = 'http://quokka2.quokkacomputing.com/qsim/qasm'
req_str_qasm = 'http://quokka1.quokkacomputing.com/qsim/qasm'

In [10]:
code = qiskit.qasm2.dumps(circuit)

# by default, qiskit includes a library of gates not installed on the Quokka
# this code removes that
code = code[:14] + code[(14 + 22):]

print(code)

OPENQASM 2.0;
qreg q[2];
creg c[2];
h q[0];
cx q[0],q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];


In [11]:
# Generate a QASM string representation of the circuit
#code = qiskit.qasm2.dumps(circuit)

# By default, Qiskit includes a library of gates not installed on Quokka.
# The following line removes the 'include "qelib1.inc";' line from the QASM string.
# This is done by taking the first 14 characters (up to and including the first newline after 'OPENQASM 2.0;')
# and then concatenating everything after the next 22 characters (which is the length of 'include "qelib1.inc";\n').
#code = code[:14] + code[(14 + 22):]

In [12]:
data = {
    'script': code,
    'count': 10
}
fy=False
result = requests.post(req_str_qasm, json=data)
json_obj = json.loads(result.content)

print(json_obj)

{'error': 'no error', 'error_code': 0, 'result': {'c': [[1, 1], [0, 0], [1, 1], [1, 1], [1, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]}}


In [13]:
bits = list(np.concatenate(list(json_obj['result'].values())).flat)

print(bits)

[np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]


In [14]:
# !pip install randtest
import randtest

randtest.random_score(bits)

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'randtest' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'randtest'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for randtest: filename=randtest-0.7-py3-none-any.whl size=2916 sha256=20d7de57a2c520abd0f335945e3462dad6458503275894d967d3a80fd1b70164
  Stored in directory: /Users/heloisamafra/Library/Caches/pip/wheels/5f/3f/1f/5036f4d83d31c5aa51759f25b02886414f67eb523a3c3f125d
Successfully built randtest


np.True_